In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC


import demjson3
import os
import json
from salaries_functions import create_country_file, write_jobs_to_txt, scrape_pages, parse_to_csv, get_netherlands_url

In [ ]:
netherlands_url = get_netherlands_url()
response = requests.get(netherlands_url)

In [ ]:
germany_url = 'https://techpays.com/europe/germany'
response = requests.get(germany_url)

In [ ]:
netherlands_url = 'https://techpays.com/europe/netherlands'
response = requests.get(netherlands_url)
create_country_file('netherlands')
create_country_file('germany')

In [ ]:
scrape_pages(netherlands_url)
scrape_pages(germany_url)

earlier netherlands iteration crashed at 129th iteration

berlin search has to stop at the 107th iteration! (added break statement)

In [ ]:
parse_to_csv('berlin')
parse_to_csv('netherlands')

In [10]:
print(df.head(5))

                          title                   guid specialization  \
0   Fullstack Software Engineer  d5m23tirj1tmu1d4lalf8                  
1  Fulllstack Software Engineer   vpm23tmk2axtoxbh9yfa                  
2             Software Engineer  ysm23tpnc0vi78l1m7rkm                  
3      Senior Software Engineer  9km23tq5471g2m3topyll                  
4             Software Engineer  ftm261a95hw8qhwi1zjmd                  

                     city companyName totalCompensation  \
0  Arnhem (flexible days)       Avisi           €56,000   
1  Arnhem (flexible days)       Avisi           €57,500   
2                  Arnhem       Avisi           €67,500   
3               Amsterdam   Cognizant           €89,500   
4               Amsterdam         Bux          €110,000   

   totalCompensationNumber   totalCompensationDetails    baseSalary  \
0                    56000  €56K salary, €1K benefits  €4,650/month   
1                    57500  €58K salary, €0K benefits  €4,806/mo

In [20]:
dutch_avg = dutch_salaries['baseSalaryNumber'].mean() / 12
print('dutch_avg',dutch_avg)

berin_avg = berlin_salaries['baseSalaryNumber'].mean() / 12
print('berin_avg',berin_avg)


dutch_avg 6608.363219949059
berin_avg 6744.639746634996


In [36]:
print(dutch_salaries.dtypes)

title                       object
guid                        object
specialization              object
city                        object
companyName                 object
totalCompensation           object
totalCompensationNumber      int64
totalCompensationDetails    object
baseSalary                  object
baseSalaryNumber             int64
oldYearForData              object
otherContext                object
dtype: object


In [35]:
for title in dutch_salaries['title'].dropna().unique():
    if 'senior' in title:
        print(title)
    if 'medior' in title:
        print(title)
    if 'junior' in title:
        print(title)

🔒 Above senior-level position within engineering management
🔒 Above senior-level position within product &amp; program management
🔒 Above senior-level position within software engineering
🔒 Above senior-level position within design
🔒 Above senior-level position within other
🔒 Above senior-level position within data science
senior software engineer
senior program manager
senior associate
Lowcode developer (medior)
junior software engineer
Consultant, Risk Advisory, Cyber Risk Services, doing mostly engineering (junior)


In [55]:

def get_page_filter_button_types_dict():
    arguments = dict.fromkeys(('locationFilterButton','jobFamilyFilterButton', 'seniorityFilterButton','roleTypeFilterButton','companyFilterButton'))
    return arguments


def get_filter_button_values(driver, filter_button):
    '''
    This functions receives a WebDriver object and a button id Object
    It returns a dictionary of all possible values for that button.
    '''
    options = {}
    button_class = re.sub(r'Button', '', filter_button) + 'Options'
    print(f'Filter button is {filter_button}, and button class is {button_class}')

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, filter_button))
    )
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    
    
    
    button.click()


    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, button_class))
    )

    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    dropdown = soup.find("div", id= button_class)
    locations = dropdown.find_all("a")

    for loc in locations:
        name = loc.text.strip()
        link = "https://techpays.com" + loc["href"]
        #print(f"{name}: {link}")
        options[name] = link
    button.send_keys(Keys.SPACE)
    return options





#########   MAIN    ####
driver = webdriver.Chrome()
driver.get(netherlands_url)
button_ids = get_page_filter_button_types_dict()
button_id_values = {button_id: {} for button_id in button_ids}

for button_id in button_ids:
    button_id_values = get_filter_button_values(driver, button_id)
    button_ids[button_id] = button_id_values
    print(button_ids[button_id])

    
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    driver.execute_script("document.querySelector('body').click();")
    time.sleep(1) 




Filter button is locationFilterButton, and button class is locationFilterOptions
{'Amsterdam': 'https://techpays.com/europe/netherlands/amsterdam', 'Outside Amsterdam': 'https://techpays.com/europe/netherlands/outside-amsterdam', 'Randstad': 'https://techpays.com/europe/netherlands/randstad', 'Almere': 'https://techpays.com/europe/netherlands/almere', 'Delft': 'https://techpays.com/europe/netherlands/delft', 'Deventer': 'https://techpays.com/europe/netherlands/deventer', 'Eindhoven': 'https://techpays.com/europe/netherlands/eindhoven', 'Enschede': 'https://techpays.com/europe/netherlands/enschede', 'Groningen': 'https://techpays.com/europe/netherlands/groningen', 'Heerlen, Limburg': 'https://techpays.com/europe/netherlands/heerlen-limburg', 'Hoofddorp': 'https://techpays.com/europe/netherlands/hoofddorp', 'Remote': 'https://techpays.com/europe/netherlands/remote', 'Rotterdam': 'https://techpays.com/europe/netherlands/rotterdam', 's-Hertogenbosch, North Brabant': 'https://techpays.com/e

In [62]:
print(json.dumps({button_id: button_ids[button_id]}, indent=2, ensure_ascii=False))


{
  "companyFilterButton": {
    "Abcam": "https://techpays.com/europe/netherlands/abcam",
    "ABN Amro": "https://techpays.com/europe/netherlands/abn-amro",
    "ABN AMRO": "https://techpays.com/europe/netherlands/abn-amro",
    "Accenture": "https://techpays.com/europe/netherlands/accenture",
    "Adevinta": "https://techpays.com/europe/netherlands/adevinta",
    "Adidas": "https://techpays.com/europe/netherlands/adidas",
    "Adyen": "https://techpays.com/europe/netherlands/adyen",
    "Ahold Delhaize": "https://techpays.com/europe/netherlands/ahold-delhaize",
    "albelli": "https://techpays.com/europe/netherlands/albelli",
    "Albert Heijn": "https://techpays.com/europe/netherlands/albert-heijn",
    "Amazon": "https://techpays.com/europe/netherlands/amazon",
    "ASML": "https://techpays.com/europe/netherlands/asml",
    "Atlassian": "https://techpays.com/europe/netherlands/atlassian",
    "Atos": "https://techpays.com/europe/netherlands/atos",
    "Avisi": "https://techpays.co

In [6]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
async def read_root():
    return 'Hello World'